## HW1 數字辨識

### 環境設置

In [67]:
# 資料處理
import numpy as np
import pandas as pd

# 常用函數
import os
import copy
import random
from tqdm import tqdm
from collections import Counter

# 影像處理函數
from PIL import Image
import matplotlib.pyplot as plt

# 分割資料
from sklearn.model_selection import train_test_split, KFold

# 資料集讀取和處理
from torchvision import transforms
from torchvision.transforms import v2
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, Dataset
from torchvision.transforms import InterpolationMode

# 模型評估矩陣
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# 深度學習架構
import torch
from   torch import nn
from   torch.optim import lr_scheduler, SGD, Adam

# 調參數
import optuna

# transforms 時需要的函式(多執行緒需要另外把函數寫在 py 檔)
import utils

In [70]:
# 創建資料夾
dataset_path = '../Dataset/'
if (not os.path.exists(dataset_path)):
    os.mkdir(dataset_path)
    print('創建 Dataset 資料夾')

result_path = '../Result/'
if (not os.path.exists(result_path)):
    os.mkdir(result_path)
    print('創建 Result 資料夾')

### 資料分析與前處理

作業給的資料集:
- noisy_train_images.npy
- noisy_train_labels.txt
- test_images.npy

為了方便之後的影像資料處理，我統一都用 numpy.ndarray 處理:
- noisy_train_images.npy -> train_x
- noisy_train_labels.txt -> train_y
- test_images.npy -> test_x

In [ ]:
# train_x 讀取
train_x = np.load('../Dataset/noisy_train_images.npy')

# train_y 讀取並寫入 npy
train_y = []
with open('../Dataset/noisy_train_labels.txt', mode='r') as file:
    lines = file.readlines()
    for line in lines:
        train_y.append(int(line.strip()))
train_y = np.array(train_y)
np.save('../Dataset/train_y.npy', train_y)

# test_x 讀取
test_x = np.load('../Dataset/test_images.npy')


# 隨機顯示 20 張影像
dataset = utils.NPDataset(train_x, train_y)

plt.figure(figsize=(20, 10))
for i in range(20):
    index = random.randint(0, len(dataset))
    img, filtered_img, label = dataset[index]
    plt.subplot(4, 10, i+1)
    plt.title(f"Label: {label}")
    plt.imshow(img)
plt.show()

- 從資料集 x 的 dimension 來看，可以知道訓練集有 60000 張，測試集有 10000 張，影像大小都是 28*28，灰階影像(通道數為1)。
- 從訓練集 y 的 dimension 來看，可以知道每個 class 的儲存方式非 one-hot，有必要在訓練時轉為 one-hot。
- 從訓練集各類別的分佈來看，可以知道數量上是蠻平均的，沒有必要做資料平衡。

In [ ]:
print('資料集 dimension:')
print(f'train_x.shape: {train_x.shape}')
print(f'train_y.shape: {train_y.shape}')
print(f'test_x.shape:  {test_x.shape}')
print('-'*30)

print('資料集分佈:')
class_distribution = [Counter(train_y)[i] for i in range(10)]
print(f'Train: {class_distribution}')
print('-'*30)

### 影像前處理

因為分類任務的資料集使用的是含椒鹽雜訊的手寫數字影像，考慮到椒鹽雜訊可以輕鬆的使用中值濾波器去雜訊，因此 MLP 的預測除了輸入原始影像外，還會輸入經過 3*3 中值濾波後的影像。

中值濾波處理我寫在 utils 裡面，NPDataset 會返回原始影像與濾波影像，其他處理都寫在 transforms.Compose 裡面。

***
- 資料集
    - 訓練集：中值濾波器濾波後，進行裁切、旋轉、縮小(不放大，因為測試集沒有放大的數字)，最後做正歸化。
    - 驗證集：為了保持影像與測試集一致，讓結果可以接近測試集，只做中值濾波器與普通的正歸化處理。
- 影像處理
    - 中值濾波器的 kernal 大小會影響到影像處理後的資訊，一般而言 kernal 越大，影像處理後所保留的資訊會越少，在經過比較和 trade off 後，採用 3\*3 大小的 kernal。(2\*2雜訊多，5\*5丟失的資訊太多)
    - 影像長寬隨機裁切 0 ~ 0.125 的比例，目的是為了平移影像，讓模型學習 "當數字不在正中心" 的情況。
    - 影像隨機旋轉 0 ~ 30 度，讓模型學習 "數字寫歪" 的情況。
    - 影像隨機(67%)縮小 1 ~ 1.5 (大小約 67%)，讓模型學習 "數字較小" 的情況。
    - 影像正歸化，幫助模型訓練時可以更好收斂。

In [ ]:
# 訓練集影像處理
train_transform = transforms.Compose([
    transforms.v2.RandomAffine(degrees=30, translate=(0.125, 0.125)),   # 隨機裁切比例 0 ~ 0.125，隨機旋轉 0 ~ 30 度
    transforms.v2.RandomZoomOut(fill=0, side_range=(1, 1.5), p=0.5),    # 隨機縮小 1 ~ 1.5 (大小約 67%)，補 0
    transforms.Resize((28, 28)),
    transforms.v2.ToTensor(),                                           # 轉 Tensor，正歸化 (/255.)
])
# 驗證集影像處理
val_transform = transforms.Compose([
    transforms.v2.ToTensor(),                                           # 轉 Tensor，正歸化 (/255.)
])

In [ ]:
# 隨機挑選 20 張影像
show_dataset = utils.NPDataset(train_x, train_y, transform=train_transform)

plt.figure(figsize=(20, 10))
for i in range(20):
    index = random.randint(0, len(show_dataset))
    img, filtered_img, label = show_dataset[index]
    plt.subplot(4, 10, i+1)
    plt.title(f"Label: {label}")
    plt.imshow(img[0])
    plt.subplot(4, 10, i+21)
    plt.title(f"Label: {label}")
    plt.imshow(filtered_img[0])
plt.show()

### 模型架構

我使用的模型需要輸入原始影像與濾波影像，各自經過 MLP 後輸出串在一起，最後經過 MLP 後不接 Sigmoid 或 Softmax 輸出 10 個類別。其中除了輸出層外的 FC 使用 RELU 激勵函數。

- original_hidden_size、filtered_hidden_size、combined_hidden_size、num_original_hidden、num_filtered_hidden 和 num_combined_hidden 由 optuna 用 貝葉斯搜尋 來找比較好的參數值。
- 不接 Sigmoid 或 Softmax，因此在決定模型預測出來的類別該分類為何時，使用的是 torch.argmax(dim=1)，相當於找這 10 個類別中最大的輸出值在哪個 arg，就預測出哪個類別。

In [ ]:
# 用 GPU 跑
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

# 模型架構
class MLP(nn.Module):
    def __init__(self,
                 input_size=28*28,
                 num_classes=10,
                 original_hidden_size=1024,
                 filtered_hidden_size=1024,
                 combined_hidden_size=1024,
                 num_original_hidden=1,
                 num_filtered_hidden=1,
                 num_combined_hidden=1):
        super(MLP, self).__init__()

        # 原始影像輸入與 MLP
        self.original_layers = nn.ModuleList()
        for i in range(num_original_hidden):
            if i == 0:
                self.original_layers.append(nn.Linear(input_size, original_hidden_size))
            else:
                self.original_layers.append(nn.Linear(original_hidden_size, original_hidden_size))
            self.original_layers.append(nn.ReLU())

        # 濾波影像輸入與 MLP
        self.filtered_layers = nn.ModuleList()
        for i in range(num_filtered_hidden):
            if i == 0:
                self.filtered_layers.append(nn.Linear(input_size, filtered_hidden_size))
            else:
                self.filtered_layers.append(nn.Linear(filtered_hidden_size, filtered_hidden_size))
            self.filtered_layers.append(nn.ReLU())

        # 結合層與 MLP
        self.combined_layers = nn.ModuleList()
        for i in range(num_combined_hidden):
            if i == 0:
                self.combined_layers.append(nn.Linear(original_hidden_size + filtered_hidden_size, combined_hidden_size))
            else:
                self.combined_layers.append(nn.Linear(combined_hidden_size, combined_hidden_size))
            self.combined_layers.append(nn.ReLU())

        # 分類
        self.fc_output = nn.Linear(combined_hidden_size, num_classes)

    def forward(self, original, filtered):
        in1 = original.reshape(len(original), -1)
        in2 = filtered.reshape(len(filtered), -1)

        # 原始影像經過 MLP
        for layer in self.original_layers:
            in1 = layer(in1)

        # 濾波影像經過 MLP
        for layer in self.filtered_layers:
            in2 = layer(in2)

        # 結合兩層的輸出後送進 MLP
        combined = torch.cat((in1, in2), dim=1)
        for layer in self.combined_layers:
            combined = layer(combined)

        # 分類
        out = self.fc_output(combined)
        return out

### 訓練函數與測試函數

- train_model 單純訓練模型，輸出 acc_history, loss_history, best_model_wts, best_acc。
- CV_train_model (5 次)交叉驗證，輸出 folds_result 包含每個 fold 的 acc_history, loss_history, best_model_wts, best_acc。
- test_model 用模型預測，輸出 pred_list, label_list。

In [35]:
def train_model(model, dataloader, criterion, optimizer, scheduler, num_epochs):
    '''
    訓練模型，返回結果。

    Parameters
    -----------
    model: :class:`__main__.MLP`
        模型本身。
    dataloader: :class:`dict` `{string: torch.utils.data.dataloader.DataLoader}`
        dataloader，應該要是一個 dict，裡面包含 train/val 的 dataloader {'train':train_dataloader, 'val':val_dataloader}。
    criterion: :class:`torch.nn.modules.loss.CrossEntropyLoss`
        Loss function。
    optimizer: :class:`torch.optim.sgd.SGD`
        優化器。
    scheduler: :class:`torch.optim.lr_scheduler.CosineAnnealingWarmRestarts`
        老實說我不知道 scheduler 要怎麼翻譯。
    num_epochs: :class:`int`
        訓練的 epochs 數。

    Returns
    -----------
    acc_history: :class:`dict` `{string: list{float}}`
        train 和 val 的 Accuracy 曲線。
    loss_history: :class:`dict` `{string: list{float}}`
        train 和 val 的 Loss 曲線。
    best_model_wts: :class:`collections.OrderedDict`
        val accuracy 最高的 model 的 state_dict。
    best_acc: :class:`float`
        最高的 val accuracy。
    '''
    acc_history = {'train' : [], 'val' : []}
    loss_history = {'train' : [], 'val' : []}
    best_acc = 0.0

    # 每一個 epoch
    for epoch in range(1, num_epochs+1):
        print(f'Epoch:{epoch}' + '-'*25)
    
        # 每一個 epoch 跑一次 train 和 val
        for phase in ['train', 'val']:

            # 根據 phase 決定模型的模式
            if phase == 'train':
                model.train()
            else:
                model.eval()

            # 計算 acc 和 loss 的變數
            running_correct = 0
            running_loss = 0.0
            totalIm = 0

            # 每一個 batch
            for img, filtered_img, _label in tqdm(dataloader[phase], total=len(dataloader[phase]), leave=False, position=0):
                
                # 把 label 轉成獨熱編碼
                label = torch.stack([nn.functional.one_hot(x, 10) for x in _label]).float()
                totalIm += len(label)

                # 單個 batch 預測結果、平均 loss 值
                img = img.to(device)
                filtered_img = filtered_img.to(device)
                label = label.to(device)
                _out = model(img, filtered_img)
                loss = criterion(_out, label)

                # 如果是 train，要反向傳播並更新權重
                if phase == 'train':
                    loss.backward()
                    optimizer.step()
                optimizer.zero_grad()

                # 把預測結果和標籤從 10 個 output 轉成單個 output
                pred = _out.argmax(dim=1)
                label = label.argmax(dim=1)
                # 計算單個 batch 的總 correct 和總 loss
                running_correct += (pred == label).sum().item()
                running_loss += loss.item() * len(label)
            
            # 計算單個 epoch 的 acc 和 loss
            epoch_acc = running_correct / totalIm
            epoch_loss = running_loss / totalIm
            acc_history[phase].append(epoch_acc)
            loss_history[phase].append(epoch_loss)

            print(f"{phase:<5}  acc.:{epoch_acc:.4f}  loss:{epoch_loss:.4f}")

            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                print('best acc. in val!')

        scheduler.step()

    return acc_history, loss_history, best_model_wts, best_acc

In [ ]:
def CV_train_model(_model, dataset, criterion, create_optimizer, create_scheduler, num_epochs=128, num_folds=5):
    '''
    交叉驗證，返回每個 Fold 的結果。

    Parameters
    -----------
    _model: :class:`__main__.MLP`
        模型本身。
    dataset: :class:`utils.NPDataset`
        繼承 Dataset 的 NPDataset，裡面要有所有的 train_x 和 train_y，在 CV_train_model 裡會再切成 Fold。
    criterion: :class:`torch.nn.modules.loss.CrossEntropyLoss`
        Loss function。
    create_optimizer: :class:`function`->`torch.optim.sgd.SGD`
        返回優化器的函數。
    create_scheduler: :class:`function`->`torch.optim.lr_scheduler.CosineAnnealingWarmRestarts`
        返回 scheduler 的函數，我還是不知道 scheduler 要怎麼翻譯。
    num_epochs: :class:`int`
        訓練的 epochs 數。
    num_folds: :class:`int`
        交叉驗證的 Fold 數量。

    Returns
    -----------
    folds_result: :class:`list`
        包含每個 fold 的：
        acc_history: :class:`dict` `{string: list{float}}`
            train 和 val 的 Accuracy 曲線。
        loss_history: :class:`dict` `{string: list{float}}`
            train 和 val 的 Loss 曲線。
        best_model_wts: :class:`collections.OrderedDict`
            val accuracy 最高的 model 的 state_dict。
        best_acc: :class:`float`
            最高的 val accuracy。
    '''
    # 資料集分割
    Kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)
    Kf_idx = Kf.split(dataset)

    folds_result = []

    # 每一個 fold
    for fold, (train_index, val_index) in enumerate(Kf_idx):
        print(f'Fold {fold + 1}')
        
        # 初始化模型
        model = copy.deepcopy(_model)
        optimizer = create_optimizer(model)
        scheduler = create_scheduler(optimizer)

        train_dataset = utils.NPDataset(train_x, train_y, train_transform)
        val_dataset = utils.NPDataset(train_x, train_y, val_transform)
        train_subsampler = torch.utils.data.SubsetRandomSampler(train_index)
        val_subsampler = torch.utils.data.SubsetRandomSampler(val_index)
        train_dataloader = DataLoader(train_dataset, batch_size=64, sampler=train_subsampler, pin_memory=True, num_workers=2)
        val_dataloader = DataLoader(val_dataset, batch_size=64, sampler=val_subsampler, pin_memory=True, num_workers=2)
        dataloader = {'train' : train_dataloader, 'val' : val_dataloader}

        acc_hist, loss_hist, model_wts, best_acc = train_model(model, dataloader, criterion, optimizer, scheduler, num_epochs)
        folds_result.append((acc_hist, loss_hist, model_wts, best_acc))

    return folds_result

In [ ]:
def test_model(model, dataloader):
    '''
    Parameters
    -----------
    model: :class:`__main__.MLP`
        模型本身。
    dataloader: :class:`torch.utils.data.dataloader.DataLoader`
        應該要是 val_dataloader。

    Returns
    -----------
    pred_list: :class:`list` `{torch.Tensor}`
        pred 的結果的 list。
    label_list: :class:`list` `{torch.Tensor}`
        label 的結果的 list。
    '''
    model.eval()

    running_correct = 0
    totalIm = 0
    pred_list = []
    label_list = []

    for img, filtered_img, label in dataloader:
        label = torch.stack([nn.functional.one_hot(x, 10) for x in label]).float()
        totalIm += len(label)
        img = img.to(device)
        filtered_img = filtered_img.to(device)
        label = label.to(device)
        out = model(img, filtered_img)

        pred = out.argmax(dim=1)
        label = label.argmax(dim=1)
        running_correct += (pred == label).sum().item()
        pred_list.extend(pred.to('cpu'))
        label_list.extend(label.to('cpu'))
    
    acc = running_correct / totalIm
    print(f"val acc.:{acc:.4f}")
    return pred_list, label_list

### 調參數

用 5-Fold Cross Validation 的 Fold 1 作為資料集，以驗證時的 Validation Accuracy 作為調參數好壞的指標，使用 optuna 調各種參數。

In [36]:
# 讀取資料集
train_x = np.load('../Dataset/noisy_train_images.npy')
train_y = np.load('../Dataset/train_y.npy')

# 用 NPDataset 從 Numpy 形式的資料讀取影像和標籤，並且在取得影像時使用 transform
train_dataset = utils.NPDataset(train_x, train_y, train_transform)
val_dataset = utils.NPDataset(train_x, train_y, val_transform)

# 分割資料集
# 只跑一個 Fold 的話，固定跑 "5-Fold Cross Validation 且 random_state=42" 的 Fold 1
Kf = KFold(n_splits=5, shuffle=True, random_state=42)
Kf_idx = Kf.split(train_dataset)
train_index, val_index = list(Kf_idx)[0]    # Fold 1 的切法
train_subsampler = torch.utils.data.SubsetRandomSampler(train_index)
val_subsampler = torch.utils.data.SubsetRandomSampler(val_index)

# 用 DataLoader 控制 batch size，用 num_workers 做多執行緒處理，pin_memory 加速資料從 CPU 移到 GPU 的速度
train_dataloader = DataLoader(train_dataset, batch_size=64, sampler=train_subsampler, pin_memory=True, num_workers=2)
val_dataloader = DataLoader(val_dataset, batch_size=64, sampler=val_subsampler, pin_memory=True, num_workers=2)
dataloader = {'train':train_dataloader, 'val':val_dataloader}

In [ ]:
# 模型固定參數
input_size = 28*28
num_classes = 10
epoch_num = 35
criterion = nn.CrossEntropyLoss()

def objective(trial):
    # 尋找較好的 learning rate
    lr = trial.suggest_loguniform('lr', 0.1, 0.9)

    # 尋找 hidden_size 和 num_hidden
    original_hidden_size = trial.suggest_int('original_hidden_size', 256, 4096)
    filtered_hidden_size = trial.suggest_int('filtered_hidden_size', 256, 4096)
    combined_hidden_size = trial.suggest_int('combined_hidden_size', 256, 4096)
    num_original_hidden = trial.suggest_int('num_original_hidden', 1, 5)
    num_filtered_hidden = trial.suggest_int('num_filtered_hidden', 1, 5)
    num_combined_hidden = trial.suggest_int('num_combined_hidden', 1, 5)

    # 創建一個新的模型
    model = MLP(input_size, num_classes, original_hidden_size, filtered_hidden_size, combined_hidden_size, num_original_hidden, num_filtered_hidden, num_combined_hidden).to(device)

    # 使用建議的參數創建一個新的優化器
    optimizer = SGD(model.parameters(), lr=lr)
    scheduler = lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=2, eta_min=0)

    # 訓練模型並返回驗證集上的準確度
    acc_hist, loss_hist, model_wts, best_acc = train_model(model, dataloader, criterion, optimizer, scheduler, epoch_num)
    return best_acc

# 創建一個學習器
study = optuna.create_study(direction='maximize')

# 優化目標函數
study.optimize(objective, n_trials=30)

In [ ]:
print("最佳的參數是：", study.best_params)
print("最佳的試驗是：", study.best_trial)

### 5-Fold 交叉驗證

我把題目給的訊練集做 5-Fold Cross Validation，除了能更準確的反映出模型的效能外，也避免資料集之間的偏差。
***
- 關於資料集分割
    - 為了避免其他人沒辦法復現出相同的結果，我統一使用 random_state=42 來打亂資料集，再進行分割。
    - 不過雖然資料集相同，在模型訓練時使用的影像處理仍然有隨機的因素，因此不保證能訓練出一模一樣的結果。

- 關於訓練參數
    - Scheduler 用 CosineAnnealingWarmRestarts，T_0=5、T_mult=3、eta_min=0，也有試過 CosineAnnealingLR，T_max=32、eta_min=0 和 StepLR，效果上來說 CosineAnnealingWarmRestarts 最好，但老實說 CosineAnnealingLR 和 StepLR 如果調的好，效果也不會差到哪裡去。
    - CosineAnnealingWarmRestarts 的退火週期公式是 (T_mult^0 + T_mult^1 + ... + T_mult^n) * T_0
    - Epoch 數量是 200，這是因為要讓 CosineAnnealingWarmRestarts 能以 T_0=5、T_mult=3 的情況下完整跑完 4 次的餘弦退火。
    - Loss Function 用 CrossEntropyLoss，沒什麼好說的，多分類任務常用的損失函數。
    - Optimizer 用 SGD，有使用過 Adam，效果差不多。

In [ ]:
# 讀取資料集
train_x = np.load('../Dataset/noisy_train_images.npy')
train_y = np.load('../Dataset/train_y.npy')

# 模型參數
epoch_num = 200

original_hidden_size = study.best_params['original_hidden_size']
filtered_hidden_size = study.best_params['filtered_hidden_size']
combined_hidden_size = study.best_params['combined_hidden_size']
num_original_hidden = study.best_params['num_original_hidden']
num_filtered_hidden = study.best_params['num_filtered_hidden']
num_combined_hidden = study.best_params['num_combined_hidden']

model = MLP(input_size, num_classes, original_hidden_size, filtered_hidden_size, combined_hidden_size, num_original_hidden, num_filtered_hidden, num_combined_hidden).to(device)

criterion = nn.CrossEntropyLoss()
def create_optimizer(model):
    return SGD(model.parameters(), lr=study.best_params['lr'])
def create_scheduler(optim):
    return lr_scheduler.CosineAnnealingWarmRestarts(optim, T_0=5, T_mult=3, eta_min=0)
    # 5 20 65 200

cv_dataset = utils.NPDataset(train_x, train_y)
folds_result = CV_train_model(model, cv_dataset, criterion, create_optimizer, create_scheduler, epoch_num, num_folds=5)

### 評估

- Accuracy曲線 和 Loss曲線
- 混淆矩陣

In [ ]:
# 畫出 acc 曲線和 loss 曲線

for fold, (acc_hist, loss_hist, model_wts, best_acc) in enumerate(folds_result):
    fig = plt.figure(figsize=(8, 3.34))
    print(f'Fold {fold+1} acc. = {best_acc:.4f}')

    for i, (name, curve) in enumerate({'Accuracy':acc_hist, 'Loss':loss_hist}.items()):
        plt.subplot(1, 2, i+1)
        plt.plot(range(epoch_num), curve['train'], label = 'train')
        plt.plot(range(epoch_num), curve['val'], label = 'val')
        plt.xlabel('Epochs')
        plt.legend()
        plt.title(f'Fold: {fold+1}  {name} Curve')

    plt.savefig(f'../Result/curve_fold{fold+1}.png')
    plt.show()

In [ ]:
# 畫出驗證集的混淆矩陣

num_folds = 5
val_dataset = utils.NPDataset(train_x, train_y, val_transform)

for fold in range(num_folds):
    Kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)
    Kf_idx = Kf.split(cv_dataset)
    train_index, val_index = list(Kf_idx)[fold]
    val_subsampler = torch.utils.data.SubsetRandomSampler(val_index)
    val_dataloader = DataLoader(val_dataset, batch_size=64, sampler=val_subsampler, pin_memory=True, num_workers=2)
    model.load_state_dict(folds_result[fold][2])
    val_pred, val_label = test_model(model, val_dataloader)

    cm = confusion_matrix(val_label, val_pred)
    disp = ConfusionMatrixDisplay(confusion_matrix = cm, display_labels = {0, 1, 2, 3, 4, 5, 6, 7, 8, 9})
    disp.plot()
    plt.savefig(f'../Result/confusion_matrix_fold{fold+1}.png')

In [52]:
# 測試集結果儲存到 CSV 檔

test_x = np.load('C:/Users/Landis/Desktop/HW1/Dataset/test_images.npy')
test_x_img = []
test_x_filtered_img = []

for i in range(test_x.shape[0]):
    img = test_x[i]
    filtered_img = np.array(utils.MedianFilter()(test_x[i]))
    blank_img = np.zeros_like(img)
    combined_image = np.stack((img, filtered_img, blank_img), axis=-1)

    transformed_image = val_transform(Image.fromarray(combined_image))
    img, filtered_img, _ = torch.split(transformed_image, 1, dim=0)

    test_x_img.append(img)
    test_x_filtered_img.append(filtered_img)
test_x_img = torch.stack(test_x_img)
test_x_filtered_img = torch.stack(test_x_filtered_img)

for fold in range(5):
    model.load_state_dict(folds_result[fold][2])

    model.eval()
    test_pred = model(test_x_img.to(device), test_x_filtered_img.to(device)).argmax(dim=1).cpu()

    df = pd.DataFrame({"SID":range(10000), "Number":test_pred})
    df.to_csv(f"../Result/result_fold{fold+1}.csv", index=False)

    # 儲存模型
    torch.save(model.state_dict(), f"../Result/result_fold{fold+1}.pt")